### NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

### Import the Template

In [ ]:
from app.templates.meat_sale.nl_template import nl_template

obs = nl_template['obligations']

print('OBLIGATIONS')
for ob in obs:
    print(f'{ob}: {obs[ob]}\n')

In [ ]:
from app.templates.meat_sale.contract_template import get_template

contract_template = get_template()

print(contract_template.to_sym())

In [ ]:
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

In [ ]:
from app.classes.spec.symboleo_spec import PAtom
from app.src.graph.graph_builder import GraphBuilder
from app.src.graph.graph_visualizer import GraphVisualizer
from app.src.dynamic_constructor import DynamicObjectConstructor

graph_builder = GraphBuilder()
graph_visualizer = GraphVisualizer()

digraph = graph_builder.build(PAtom)

gv = graph_visualizer.create_viz(digraph.nodes)
gv.show('nx.html')

dynamic_constructor = DynamicObjectConstructor(digraph)


### Reusable

In [ ]:
from app.src.rules.shared.predicate_processor import PredicateProcessor
from app.src.rules.shared.domain_prop_processor import DomainPropProcessor

from app.src.rules.shared.configs import PredicateExtractorConfig, PredicateProcessorConfig
from app.src.rules.shared.configs import DomainPropProcessorConfig
from app.src.rules.shared.predicate_extractor import PredicateExtractor

In [ ]:
from app.src.norm_proposition_updater import NormPropositionUpdater
from app.src.dynamic_constructor import DynamicObjectConstructor

dynamic_constructor = DynamicObjectConstructor(digraph)
norm_updater = NormPropositionUpdater()

### DELIVERY_TIMEFRAME

In [ ]:
from app.src.rules.meat_sale.delivery_timeframe.dtf_config import get_dtf_pred_config

dtf_pred_proc_config = PredicateProcessorConfig(
    norm_type = 'obligations',
    norm_id = 'delivery',
    norm_component = 'consequent',
)

dtf_predex_config = get_dtf_pred_config(nlp)

dtf_predicate_extractor = PredicateExtractor(nlp, dtf_predex_config, dynamic_constructor)

dtf_processor = PredicateProcessor(dtf_pred_proc_config, dtf_predicate_extractor, norm_updater)

### DELIVERY_LOCATION

In [ ]:
from app.src.rules.meat_sale.delivery_location.dl_libs import get_dl_libs

dl_libs = get_dl_libs(nlp)
dl_location_extractor = dl_libs['location_extractor']

dl_config = DomainPropProcessorConfig(
    'events',
    'delivered',
    'delDueDate',
    'str'
)

dl_processor = DomainPropProcessor(dl_config, dl_location_extractor)

### PAYMENT_AMOUNT

In [ ]:
from app.src.rules.meat_sale.payment_amount.pa_libs import get_pa_libs

pa_libs = get_pa_libs(nlp)
pa_currency_extractor = pa_libs['currency_extractor']
pa_amount_extractor = pa_libs['amount_extractor']

pa_curr_config = DomainPropProcessorConfig(
    'events',
    'paid',
    'currency',
    'str'
)

pa_amt_config = DomainPropProcessorConfig(
    'events',
    'paid',
    'amount',
    'str'
)

pa_curr_processor = DomainPropProcessor(pa_curr_config, pa_currency_extractor)
pa_amt_processor = DomainPropProcessor(pa_amt_config, pa_amount_extractor)

### PAYMENT_TIMEFRAME

In [ ]:
from app.src.rules.meat_sale.payment_timeframe.ptf_config import get_ptf_pred_config

ptf_pred_proc_config = PredicateProcessorConfig(
    norm_type = 'obligations',
    norm_id = 'payment',
    norm_component = 'consequent',
)

ptf_predex_config = get_ptf_pred_config(nlp, contract_template)
ptf_predicate_extractor = PredicateExtractor(nlp, ptf_predex_config, dynamic_constructor)
ptf_processor = PredicateProcessor(ptf_pred_proc_config, ptf_predicate_extractor, norm_updater)

### INTEREST_AMOUNT

In [ ]:
from app.src.rules.meat_sale.interest_amount.ia_libs import get_ia_libs

ia_libs = get_ia_libs(nlp)
ia_currency_extractor = ia_libs['currency_extractor']
ia_amount_extractor = ia_libs['amount_extractor']

ia_curr_config = DomainPropProcessorConfig(
    'events',
    'paidLate',
    'currency',
    'str'
)

ia_amt_config = DomainPropProcessorConfig(
    'events',
    'paidLate',
    'amount',
    'str'
)

ia_curr_processor = DomainPropProcessor(ia_curr_config, ia_currency_extractor)
ia_amt_processor = DomainPropProcessor(ia_amt_config, ia_amount_extractor)

### Contract Updater

In [ ]:
from app.src.contract_updater import ContractUpdater, ContractUpdateRequest
from app.src.processor_lookup import ProcessorLookup

processor_dict = {
    'DELIVERY_TIMEFRAME': [dtf_processor],
    'DELIVERY_LOCATION': [dl_processor],
    'PAYMENT_AMOUNT': [pa_curr_processor, pa_amt_processor], 
    'PAYMENT_TIMEFRAME': [ptf_processor],
    'INTEREST_AMOUNT': [ia_amt_processor, ia_curr_processor]
}
processor_lookup = ProcessorLookup(processor_dict)

contract_updater = ContractUpdater(processor_lookup)

contract = contract_template

In [ ]:
customizations = [
    ('DELIVERY_TIMEFRAME', 'within 2 weeks'),
    ('DELIVERY_LOCATION', 'at the buyer\'s warehouse'),
    ('PAYMENT_AMOUNT', '$100.00 CAD'),
    ('PAYMENT_TIMEFRAME', 'before April 17, 2022'),
    ('INTEREST_AMOUNT', '10% of the amount owed')
]

In [ ]:
for c in customizations:
    key = c[0]
    value = c[1]
    doc = nlp(value)

    req = ContractUpdateRequest(contract, key, value, doc)

    print(key)
    contract = contract_updater.update(req)

In [ ]:
print(contract.to_sym())